In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta
from dkelectricity import DKElectricity

# Put your token from eloverblik.dk here
token = ''

# Or read it from a file
if token == '':
  with open('eloverblik.token', 'r') as f:
    token = f.read()

el = DKElectricity(token)

In [ ]:
# Show data for the past weeks
weeks = 1

from_date = datetime.today() + timedelta(weeks=-weeks, days=1)
to_date = datetime.today() + timedelta(days=1) # One day into the future to get next day prices

# Add one day, as both eloverblik and energidataservice APIs don't include end date
to_date += timedelta(days=1)

In [ ]:
# Electricity prices (average over weeks)

prices = el.get_prices(from_date, to_date)

# Add some usefull columns
prices['Date'] = prices.index.date
prices['Hour'] = prices.index.hour
prices['Weekday'] = prices.index.dayofweek

# Separate last day (probably the future)
last_date = prices['Date'].iloc[-1]
price_last = prices[prices['Date'] == last_date]
prices.drop(price_last.index, inplace=True)
price_last.set_index('Hour', inplace=True)

ax = prices.groupby(['Hour'])['Price'].mean().plot.bar(label='Avg', color='lightgray', figsize=(10,6))

ax.hlines(prices['Price'].mean(), -1, 24, color='gray', label='Avg')

color = np.flipud(plt.cm.gist_rainbow(np.linspace(0, 1, 7)))

for weekday, price in prices[['Weekday', 'Hour', 'Price']].groupby(['Hour', 'Weekday']).mean().groupby('Weekday'):
  ax.plot(price.index.get_level_values('Hour').values,
          price['Price'],
          label=calendar.day_abbr[weekday],
          marker='o',
          color=color[weekday])

ax.plot(price_last.index.get_level_values('Hour').values,
        price_last['Price'],
        label=last_date.strftime('%-d/%-m'),
        marker='o',
        color='k',
        linewidth=3)

el.tariffs['Tariff'].plot.bar(label='Tariffs', color='darkgray')

ax.legend()
ax.set_title(f"Electricity prices, Avg: {prices['Price'].mean():.2f} DKK/kWh, {prices['Date'].iloc[0].strftime('%Y-%m-%d')} - {prices['Date'].iloc[-1].strftime('%Y-%m-%d')}")
ax.set_ylabel('Price [DKK/kWh]')
ax.grid(axis='y')
ax.tick_params(axis='x', labelrotation = 90)

In [ ]:
# Electricity consumption

consumption = el.get_consumption(from_date, to_date)

# Add some usefull columns
consumption['Date'] = consumption.index.date
consumption['Hour'] = consumption.index.hour
consumption['Weekday'] = consumption.index.dayofweek

ax = consumption[['Weekday', 'Hour', 'Consumption']].groupby('Hour')['Consumption'].mean().plot.bar(label='Avg', color='lightgray', figsize=(10,6))

ax.hlines(consumption['Consumption'].mean(), -1, 24, color='gray', label='Avg')

color = np.flipud(plt.cm.gist_rainbow(np.linspace(0, 1, 7)))

for weekday, consump in consumption[['Weekday', 'Hour', 'Consumption']].groupby(['Hour', 'Weekday']).mean().groupby('Weekday'):
  ax.plot(consump.index.get_level_values('Hour').values,
          consump['Consumption'],
          label=calendar.day_abbr[weekday],
          marker='o',
          color=color[weekday])

ax.set_title(f"Consumption @ {el.address}, Avg: {consumption['Consumption'].mean()*24:.1f} kWh/day, {consumption.index[0].strftime('%Y-%m-%d')} - {consumption.index[-1].strftime('%Y-%m-%d')}")
ax.set_ylabel('Consumption [kWh]')
ax.legend()
ax.grid(axis='y')